In [ ]:
begin_hour, static_break_rate = 0, 0.5

hist_day = hist_hour.resample('1D', offset=datetime.timedelta(hours=begin_hour)).agg({
    'open': 'first', #lambda df: None if df.empty else df[0],
    'close': 'last', #lambda df: None if df.empty else df,
    'high': 'max',
    'low': 'min',
    'volume': 'sum',
    'value': 'sum',
})

# common
hist_day['ma3_open'] = hist_day['open'].shift(1).rolling(3).mean()
hist_day['ma5_open'] = hist_day['open'].shift(1).rolling(5).mean()
hist_day['ma10_open'] = hist_day['open'].shift(1).rolling(10).mean()
hist_day['ma3_volume'] = hist_day['volume'].shift(1).rolling(3).mean()
hist_day['ma5_volume'] = hist_day['volume'].shift(1).rolling(5).mean()
hist_day['ma10_volume'] = hist_day['volume'].shift(1).rolling(10).mean()
hist_day['range_prev'] = hist_day['high'].shift(1) - hist_day['low'].shift(1)
hist_day['breakout_price_static'] = hist_day['open'] + hist_day['range_prev'] * static_break_rate
hist_day['breakout'] = hist_day['high'] > hist_day['breakout_price_static']

days = len(hist_day.index)


# Naive Hold
hist_day['Naive.rate_of_return'] =  hist_day['close'] / hist_day['open']
hist_day['Naive.assets_after_sell'] = hist_day['Naive.rate_of_return'].cumprod()
hist_day['Naive.assets_max'] = hist_day['Naive.assets_after_sell'].cummax()
hist_day['Naive.dd'] = 1 - hist_day['Naive.assets_after_sell'] / hist_day['Naive.assets_max']
hist_day['Naive.rate_of_return_percent'] = (hist_day['Naive.rate_of_return'] - 1) * 100
hist_day['Naive.rate_of_return_percent_profit'] = np.where(hist_day['Naive.rate_of_return_percent'] > 0, hist_day['Naive.rate_of_return_percent'], np.nan)
hist_day['Naive.rate_of_return_percent_loss'] = np.where(hist_day['Naive.rate_of_return_percent'] < 0, -hist_day['Naive.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "Naive.final_asset" : hist_day['Naive.assets_after_sell'][-1],
    "Naive.cagr(%)" : hist_day['Naive.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "Naive.mdd(%)" : np.max(hist_day['Naive.dd']) * 100,
    "Naive.ratio_win(%)" : np.sum(np.where(hist_day['Naive.rate_of_return'] > 1, 1, 0)) / len(hist_day['Naive.rate_of_return']) * 100,
    "Naive.mean_profit(%)" : np.nanmean(hist_day['Naive.rate_of_return_percent_profit']),
    "Naive.mean_loss(%)" : np.nanmean(hist_day['Naive.rate_of_return_percent_loss']),
})


# LWVB (classic)
hist_day['LWVB.rate_of_return'] = np.where(hist_day['breakout'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1) # breakout 안했으면 자산 유지
hist_day['LWVB.assets_after_sell'] = hist_day['LWVB.rate_of_return'].cumprod()
hist_day['LWVB.assets_max'] = hist_day['LWVB.assets_after_sell'].cummax()
hist_day['LWVB.dd'] = 1 - hist_day['LWVB.assets_after_sell'] / hist_day['LWVB.assets_max']
hist_day['LWVB.rate_of_return_percent'] = (hist_day['LWVB.rate_of_return'] - 1) * 100
hist_day['LWVB.rate_of_return_percent_profit'] = np.where(hist_day['LWVB.rate_of_return_percent'] > 0, hist_day['LWVB.rate_of_return_percent'], np.nan)
hist_day['LWVB.rate_of_return_percent_loss'] = np.where(hist_day['LWVB.rate_of_return_percent'] < 0, -hist_day['LWVB.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB.final_asset" : hist_day['LWVB.assets_after_sell'][-1],
    "LWVB.cagr(%)" : hist_day['LWVB.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB.mdd(%)" : np.max(hist_day['LWVB.dd']) * 100,
    "LWVB.ratio_win(%)" : np.sum(np.where(hist_day['LWVB.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['breakout'], 1, 0)) * 100,
    "LWVB.mean_profit(%)" : np.nanmean(hist_day['LWVB.rate_of_return_percent_profit']),
    "LWVB.mean_loss(%)" : np.nanmean(hist_day['LWVB.rate_of_return_percent_loss']),
})

# LWVB with MT(ma5_open)
hist_day['LWVB_MT1.is_bull'] = (hist_day['breakout']) & (hist_day['open'] > hist_day['ma5_open'])
hist_day['LWVB_MT1.rate_of_return'] = np.where(hist_day['LWVB_MT1.is_bull'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1) 
hist_day['LWVB_MT1.assets_after_sell'] = hist_day['LWVB_MT1.rate_of_return'].cumprod()
hist_day['LWVB_MT1.assets_max'] = hist_day['LWVB_MT1.assets_after_sell'].cummax()
hist_day['LWVB_MT1.dd'] = 1 - hist_day['LWVB_MT1.assets_after_sell'] / hist_day['LWVB_MT1.assets_max']
hist_day['LWVB_MT1.rate_of_return_percent'] = (hist_day['LWVB_MT1.rate_of_return'] - 1) * 100
hist_day['LWVB_MT1.rate_of_return_percent_profit'] = np.where(hist_day['LWVB_MT1.rate_of_return_percent'] > 0, hist_day['LWVB_MT1.rate_of_return_percent'], np.nan)
hist_day['LWVB_MT1.rate_of_return_percent_loss'] = np.where(hist_day['LWVB_MT1.rate_of_return_percent'] < 0, -hist_day['LWVB_MT1.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB_MT1.final_asset" : hist_day['LWVB_MT1.assets_after_sell'][-1],
    "LWVB_MT1.cagr(%)" : hist_day['LWVB_MT1.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB_MT1.mdd(%)" : np.max(hist_day['LWVB_MT1.dd']) * 100,
    "LWVB_MT1.ratio_win(%)" : np.sum(np.where(hist_day['LWVB_MT1.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['LWVB_MT1.is_bull'], 1, 0)) * 100,
    "LWVB_MT1.mean_profit(%)" : np.nanmean(hist_day['LWVB_MT1.rate_of_return_percent_profit']),
    "LWVB_MT1.mean_loss(%)" : np.nanmean(hist_day['LWVB_MT1.rate_of_return_percent_loss']),
})
    
# LWVB_MT2
hist_day['LWVB_MT2.is_bull'] = (hist_day['breakout']) & (hist_day['open'] > hist_day['ma5_open']) & (hist_day['volume'] > hist_day['ma3_volume']) 
hist_day['LWVB_MT2.rate_of_return'] = np.where(hist_day['LWVB_MT2.is_bull'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1) 
hist_day['LWVB_MT2.assets_after_sell'] = hist_day['LWVB_MT2.rate_of_return'].cumprod()
hist_day['LWVB_MT2.assets_max'] = hist_day['LWVB_MT2.assets_after_sell'].cummax()
hist_day['LWVB_MT2.dd'] = 1 - hist_day['LWVB_MT2.assets_after_sell'] / hist_day['LWVB_MT2.assets_max']
hist_day['LWVB_MT2.rate_of_return_percent'] = (hist_day['LWVB_MT2.rate_of_return'] - 1) * 100
hist_day['LWVB_MT2.rate_of_return_percent_profit'] = np.where(hist_day['LWVB_MT2.rate_of_return_percent'] > 0, hist_day['LWVB_MT2.rate_of_return_percent'], np.nan)
hist_day['LWVB_MT2.rate_of_return_percent_loss'] = np.where(hist_day['LWVB_MT2.rate_of_return_percent'] < 0, -hist_day['LWVB_MT2.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB_MT2.final_asset" : hist_day['LWVB_MT2.assets_after_sell'][-1],
    "LWVB_MT2.cagr(%)" : hist_day['LWVB_MT2.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB_MT2.mdd(%)" : np.max(hist_day['LWVB_MT2.dd']) * 100,
    "LWVB_MT2.ratio_win(%)" : np.sum(np.where(hist_day['LWVB_MT2.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['LWVB_MT2.is_bull'], 1, 0)) * 100,
    "LWVB_MT2.mean_profit(%)" : np.nanmean(hist_day['LWVB_MT2.rate_of_return_percent_profit']),
    "LWVB_MT2.mean_loss(%)" : np.nanmean(hist_day['LWVB_MT2.rate_of_return_percent_loss']),
})

# LWVB_V : LWVB with volatility control
hist_day['LWVB_V.is_bull'] = (hist_day['breakout']) & (hist_day['open'] > hist_day['ma5_open']) & (hist_day['volume'] > hist_day['ma3_volume'])
hist_day['LWVB_V.volatility'] = hist_day['range_prev'] / hist_day['open'].shift(1)
hist_day['LWVB_V.invest_ratio'] = np.where(hist_day['LWVB_V.volatility'] <= 0.01, 1, 0.01 / hist_day['LWVB_V.volatility'])
hist_day['LWVB_V.rate_of_return'] = \
    (1 - hist_day['LWVB_V.invest_ratio']) + \
        hist_day['LWVB_V.invest_ratio'] * \
            np.where(hist_day['LWVB_V.is_bull'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1)
hist_day['LWVB_V.assets_after_sell'] = hist_day['LWVB_V.rate_of_return'].cumprod()
hist_day['LWVB_V.assets_max'] = hist_day['LWVB_V.assets_after_sell'].cummax()
hist_day['LWVB_V.dd'] = 1 - hist_day['LWVB_V.assets_after_sell'] / hist_day['LWVB_V.assets_max']
hist_day['LWVB_V.rate_of_return_percent'] = (hist_day['LWVB_V.rate_of_return'] - 1) * 100
hist_day['LWVB_V.rate_of_return_percent_profit'] = np.where(hist_day['LWVB_V.rate_of_return_percent'] > 0, hist_day['LWVB_V.rate_of_return_percent'], np.nan)
hist_day['LWVB_V.rate_of_return_percent_loss'] = np.where(hist_day['LWVB_V.rate_of_return_percent'] < 0, -hist_day['LWVB_V.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB_V.final_asset" : hist_day['LWVB_V.assets_after_sell'][-1],
    "LWVB_V.cagr(%)" : hist_day['LWVB_V.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB_V.mdd(%)" : np.max(hist_day['LWVB_V.dd']) * 100,
    "LWVB_V.ratio_win(%)" : np.sum(np.where(hist_day['LWVB_V.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['LWVB_V.is_bull'], 1, 0)) * 100,
    "LWVB_V.mean_profit(%)" : np.nanmean(hist_day['LWVB_V.rate_of_return_percent_profit']),
    "LWVB_V.mean_loss(%)" : np.nanmean(hist_day['LWVB_V.rate_of_return_percent_loss']),
})

px.line(
    hist_day, 
    y=[
        'Naive.assets_after_sell',
        'LWVB.assets_after_sell',
        'LWVB_MT1.assets_after_sell',
        'LWVB_MT2.assets_after_sell',
        'LWVB_V.assets_after_sell'], 
    log_y=True).show()
px.line(
    hist_day, 
    y=[
        'Naive.dd',
        'LWVB.dd',
        'LWVB_MT1.dd',
        'LWVB_MT2.dd',
        'LWVB_V.dd']
    ).show()


hist_day.to_csv(targetDataFile + "_result_LWVB.csv")